# Hospital Staff Scheduling Optimization  
**Course:** Optimization and Analytics — Bachelor in Data Science and Engineering, UC3M  
**Students:** Fenz & Kahlkopf  
**Deadline:** November 16, 2025

---

# Introduction
In this project, we develop a **linear and mixed-integer optimization model** for the weekly scheduling of hospital staff.  
The goal is to assign doctors, nurses, and other healthcare workers to morning, afternoon, and night shifts while minimizing total staffing costs and respecting multiple operational and human constraints.

The dataset `hospital_staff.csv` contains information about 20 employees, including:
- Role, gender, and experience level  
- Maximum weekly working hours  
- Hourly cost  
- Individual availability and shift preferences  

This model helps hospital administrators create efficient and fair schedules that:
- Guarantee the required number of staff in every shift,  
- Respect employees’ availability and workload limits,  
- Avoid infeasible or unfair assignments,  
- Minimize the total labor cost.

In part (a), we formulate the **linear programming (LP)** version of the problem.  
In part (b), we implement and solve the model using realistic data.  
In part (c), we analyze **sensitivities (shadow prices)** to understand how changes in constraints affect costs.  
Finally, in part (d), we extend the model to a **mixed-integer programming (MIP)** version by adding logical and conditional constraints such as rest periods after night shifts and maximum weekly work limits.


## (a) Linear Optimization Model — General Formulation

### Objective
The goal is to assign hospital staff members (doctors, nurses, interns, technicians, and instructors)  
to morning, afternoon, and night shifts over a weekly horizon **while minimizing total labor cost** and  
respecting individual availability, workload, and staffing requirements.

---

### Sets
| Symbol | Description |
|:--------|:-------------|
| \( I \) | Set of all staff members \( i \in I \) (20 employees) |
| \( D \) | Set of days in the planning horizon (e.g., Monday–Sunday) |
| \( S \) | Set of shifts per day (Morning, Afternoon, Night) |

---

### Parameters
| Symbol | Description |
|:--------|:-------------|
| \( c_{i,s} \) | Cost of assigning staff \( i \) to shift \( s \) (€/hour) |
| \( h_s \) | Duration of shift \( s \) (hours) |
| \( a_{i,s} \in \{0,1\} \) | Availability of staff \( i \) for shift \( s \) (1 = available, 0 = not available) |
| \( H_i \) | Maximum working hours per week for staff \( i \) |
| \( R_{s,d} \) | Minimum number of required staff members for shift \( s \) on day \( d \) |

---

### Decision Variables
| Symbol | Description |
|:--------|:-------------|
| \( x_{i,s,d} \in [0,1] \) | 1 if staff member \( i \) works on day \( d \) in shift \( s \); 0 otherwise |

In the **LP model**, \( x_{i,s,d} \) can take continuous values in [0,1].  
In the **MIP extension**, it will become binary (\( x_{i,s,d} \in \{0,1\} \)).

---

### Objective Function
Minimize total labor cost over all staff, shifts, and days:

$$
\text{Minimize } Z = \sum_{i \in I} \sum_{s \in S} \sum_{d \in D} c_{i,s} \, h_s \, x_{i,s,d}
$$

---

### Constraints

1. **Shift coverage requirement:**  
   Each shift on each day must be covered by the required number of staff:
   $$
   \sum_{i \in I} x_{i,s,d} \ge R_{s,d} \quad \forall s \in S, \, d \in D
   $$

2. **At most one shift per person per day:**  
   A staff member can only work one shift per day:
   $$
   \sum_{s \in S} x_{i,s,d} \le 1 \quad \forall i \in I, \, d \in D
   $$

3. **Weekly working hour limit:**  
   Each staff member cannot exceed their maximum weekly hours:
   $$
   \sum_{s \in S} \sum_{d \in D} h_s \, x_{i,s,d} \le H_i \quad \forall i \in I
   $$

4. **Availability constraint:**  
   Staff can only be assigned to shifts they are available for:
   $$
   x_{i,s,d} \le a_{i,s} \quad \forall i \in I, \, s \in S, \, d \in D
   $$

5. **Non-negativity:**  
   $$
   x_{i,s,d} \ge 0
   $$

---

### Model Summary
This linear programming formulation captures a realistic hospital scheduling scenario where the hospital must:
- Ensure adequate coverage per shift and day,  
- Respect each employee’s workload and availability,  
- Minimize total operational cost.

In [1]:
import pandas as pd

staff = pd.read_csv("hospital_staff.csv")
staff.head()


,staff_id,name,gender,role,experience_level,max_hours_per_week,cost_per_hour,shift_preference,availability_morning,availability_afternoon,availability_night
0,1,Alice Johnson,Male,Intern,Mid,27,11,Morning,1,1,0
1,2,Bob Smith,Male,Technician,Mid,30,18,Morning,1,1,0
2,3,Clara Lee,Male,Nurse,Mid,45,28,Afternoon,0,0,0
3,4,David Brown,Female,Nurse,Senior,35,22,Night,1,1,1
4,5,Eva Green,Female,Nurse,Junior,40,21,Morning,0,1,1
